In [83]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
import math
import warnings
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

from IPython.display import display

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
from sklearn import neighbors

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif

from timeit import default_timer as timer

import pydotplus
from sklearn import tree
from sklearn import preprocessing
%matplotlib inline
sns.set_style('white')

In [109]:
# Load dataset
raw_data = pd.read_csv('epi_r.csv')


In [111]:
drop_na = raw_data.dropna()
drop_na['highly_rated'] = (drop_na['rating'] > 3)*1
nutrition_only = drop_na.loc[:,['title', 'rating', 'calories', 'protein', 'fat', 'sodium', 'highly_rated']]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [115]:
# Picking 5 SelectKBest features
X = drop_na.drop(['rating', 'title', 'highly_rated', 'calories','protein','fat','sodium'], 1)
Y = drop_na['highly_rated']

X_best_selection = SelectKBest(f_classif, k=5).fit_transform(X, Y)

X_best_df = pd.DataFrame({
    'best_1':X_best_selection[:,0],
    'best_2':X_best_selection[:,1],
    'best_3':X_best_selection[:,2],
    'best_4':X_best_selection[:,3],
    'best_5':X_best_selection[:,4]})


X_best = pd.concat([nutrition_only, X_best_df], axis = 1)
X_best = X_best.dropna()

/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 93 195 221 225 243 288 294 319 381 397 524 643] are constant.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [116]:
test_df = X_best

svc = SVC()
X = test_df.drop(['rating', 'title', 'highly_rated'], 1)
Y = test_df.highly_rated
svc.fit(X,Y)

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [117]:
model_score = svc.score(X, Y)
print('Model score: ' + str(model_score))
print('Cross Valuation: ' + str(cross_val_score(svc, X, Y, cv=5)))

Model score: 0.9793503296005083


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

Cross Valuation: [0.89162366 0.89202064 0.88959492 0.89078634 0.88994835]


Setting the model to estimate ratings of >=4 (making that a binary variable) worked quite well in terms of producing an accurate outcome, but... I'm not certain how the cross-valuation scores can be so abysmal while the score for the model itself is in th %90+ range.  

In [118]:
test_df.head(2)

,title,rating,calories,protein,fat,sodium,highly_rated,best_1,best_2,best_3,best_4,best_5
0,"Lentil, Apple, and Turkey Wrap",2.500,426.000,30.000,7.000,559.000,0.000,0.000,0.000,0.000,0.000,0.000
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.000,18.000,23.000,1439.000,1.000,0.000,0.000,0.000,0.000,0.000
